 <font size="6">Генеративные модели</font>

# "Классические" генеративные алгоритмы

## Задача генерации

В этом курсе мы в основном работали с **размеченными** данными. Мы научили нейронные сети решать задачи классификации, регрессии, сегментации и т.д. На входе модели у нас были исходные данные по которым мы учились предсказывать целевые значения (метки классов, действительные числа, маски сегментации и т.д):

$$\large \text{Воходные данные} \xrightarrow[\text{}]{\text{Модель}} \text{Целевые значения}$$

Математически это можно записать, как

$$\large X \xrightarrow[\text{}]{f_{θ}(x)} Y$$
где $X$ - множество входных данных, $Y$ - множество целевых значений, $f_{θ}(x)$ - модель с настраиваемыми параметрами $θ$. По сути наша модель - это сложная функция, которая ставит в соответствие каждому объекту $x$ из множества входных данных $X$ целевое значение $y$ из множества выходных данных $Y$:
$$y = f_{θ}(x)$$


С задачей генерации дела обстоят сложнее, ведь в задаче генерации мы хотим научить модель генерировать объекты, которых не было в исходной выборке.

**Постановка задачи генерации**

**Дано**: данные, часто неразмеченные.

**Вход модели**: ?????

**Результат генерации**: новые данные, которые будут удовлетворять следующим условиям:
* Новые данные **похожи** на исходные.
* Новые данные **не повторяют** исходные (или повторяют, очень редко, при случайном стечении обстоятельств).

**Дополнительно:**
* Важен **элемент случайности**: результат генерации не повторяется при разных запусках.

Итого не очень понятно, **что должно подаваться на вход модели** и как обеспечить элемент случайности. В большинстве моделей генерации в качестве входных данных для модели используют **гауссовский или равномерный шум.**

$$\large \text{Шум} \xrightarrow[\text{}]{\text{Модель генерации}} \text{Cгенеренный объект}$$

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/generator_model_pipeline.png" width="350"></center>

Математически это можно записать, как

$$\large Z \xrightarrow[\text{}]{g_{θ}(z)} X$$
где $Z$ - хорошо изученное множество (например, нормальне гауссово распреление) из которого мы умеем семплировать объекты, $X$ - множество всех возможных объектов с неизвестным распределением или известным распределением из которого мы не умеем семплировать.

По сути модель генерации - это сложная функция, которая ставит в соответствие выборке из известного распеделения $z$ объект из целевого распределения $x$:
$$x = g_{θ}(z)$$


<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/generative_models.png" width="700">

## Простейший пример: генерация объектов из нормального распределения

Чтобы обеспечить воспроизводимость примеров - установим сиды:

In [ ]:
import random
import torch
import numpy as np

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

Предположим, у нас под рукой есть генератор случайных чисел, который позволяет нам легко получить случайные числа в диапазоне $[0,1]$:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

u1 = np.random.uniform(size=1000)
u2 = np.random.uniform(size=1000)

data = pd.DataFrame.from_dict(dict(zip(["U1", "U2"], [u1, u2])))

sns.jointplot(data=data, x="U1", y="U2")
plt.show()

Пусть $U_i$ — точка, которую даёт нам генератор равномерно распределенных случайных чисел. Если мы возьмём пару таких таких точек $U_1$, $U_2$ и произведём с ними так называемое [преобразование Бокса-Мюллера 📚[wiki]](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform), то получим пару новых точек $X$ и $Y$ на вещественной прямой:

$$
\begin{array}{l}
X=\sqrt{-2 \log U_{1}} \cos \left(2 \pi U_{2}\right) \\
Y=\sqrt{-2 \log U_{1}} \sin \left(2 \pi U_{2}\right)
\end{array}
$$

Повторив такую процедуру для большого числа точек $U_i$, можно заметить, что распределение для объектов $X$ и $Y$ становится похожим на стандартное нормальное:

In [ ]:
r = np.sqrt(-2 * np.log(u1))
theta = 2 * np.pi * u2

x = r * np.cos(theta)
y = r * np.sin(theta)

data["X"] = x
data["Y"] = y

sns.jointplot(data=data, x="X", y="Y")
plt.show()

На самом деле мы можем в точности доказать, что преобразование Бокса-Мюллера преобразовало исходное равномерное распределение в стандартное нормальное. С доказательством можно ознакомиться [тут](https://mathworld.wolfram.com/Box-MullerTransformation.html).

Можно раскрасить точки выборки и посмотреть, как именно они преобразуются:

In [ ]:
color = np.sqrt(u1*u1 + u2*u2) # for visualization
color = color / color.max()
data["color"] = color

f, ax = plt.subplots(1, 2, figsize=(14, 6))

cmap = sns.color_palette("Spectral", as_cmap=True)
sns.scatterplot(
    data=data, x="U1", y="U2", hue="color", palette=cmap, ax=ax[0],
    legend=False,
)
sns.scatterplot(
    data=data, x="X", y="Y", hue="color", palette=cmap, ax=ax[1], legend=False
)
plt.show()

Что делать, если нам хочется сгенерировать объекты не в стандартном нормальном распределении, а в каком-то более сложном? В общем случае нам необходимо подобрать некоторую функцию, которая будет отображать объекты из известного "простого" распределения в "целевое". Осуществляющие такое преобразование функции будем называть **генеративными моделями**.

Если целевое распределение $\mathcal{F}[\vec {x}]$ известно аналитически, то для подбора такой функции можно воспользоваться различными модификациями алгоритма [Markov Chain Monte Carlo 📚[wiki]](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo).

# Генеративные алгоритмы, основанные на глубоком обучении

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/deep_generative_models.png" width="700">

# Введение  в генеративно-состязательные нейронные сети


[Множество примеров различных генераторов](https://thisxdoesnotexist.com)


## Наивный подход в решении задачи генерации

## Latent space

Разберемся с **элементом случайности**. В нейронных сетях мы привыкли к **воспроизводимости** результата: в режиме валидации мы можем несколько раз подать на вход один и тот же объект и получить один и тот результат.  Возникает два вопроса:
- что подавать на **вход** сети для генерации?
- как реализовать **случайность**?

Ответ на оба вопроса: подавать в качестве **входа** вектор **случайного шума**.

Почему именно **вектор**? Почему не одно **случайное число**?

**Ответ**: входной вектор можно рассматривать как **признаки** генерируемого объекта. Каждый такой признак — **независимая случайная величина**. Если мы будем передавать только одно случайное число, то генерация будет однообразной. Чем больше признаков (степеней свободы) у входного вектора, тем разнообразнее будет результат генерации.

То есть случайный шум **большей размерности** даёт нам **больше вариабельности**  для генерации. Это называется **input latent space** — входное латентное пространство.

**Note:** *из-за неустоявшейся терминологии случайное распределение на входе генератора называется латентным пространством так же, как и скрытое пространство в автоэнкодерах. Поэтому в этой лекции будем называть его **входным** латентным пространством. Также в статьях встречается вариант: predefined latent space.*

Каждую компоненту латентного пространства можно рассматривать как отдельную шкалу, вдоль которой изменяются определенные свойства генерируемых объектов. Например, можно выбрать четыре латентных вектора и посмотреть, как генерируемые объекты плавно изменяются при переходе от одного вектора к другому.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L13/input_latent_space_lin_interpol.png" width="600">

<center><em>Линейные интерполяции между четырьмя изображениями в латентном пространстве</em></center>

<center><em>Source: <a href="https://arxiv.org/abs/1803.09093">Comparing Generative Adversarial Network Techniques for Image Creation and Modification(M. Pieters, M. Wiering, 2018)</a></em></center>

### Размерность входного латентного пространства

В выборе размерности входного латентного пространства важно соблюсти  баланс.
- при **низкой размерности** возникнет проблема **низкой вариабельности**.

Пример: генератор лиц с входным вектором длины 1. Результатом работы генератора будет всего одна шкала, вдоль которой будут расположены генерируемые изображения. Сеть скорее всего **не сможет выдавать все многообразие объектов**, она выучит небольшую часть: например при изменении значений будет выдавать от молодой женщины блондинки до пожилого мужчины брюнета. У такой сети будет **низкая вариабельность** — она не сможет сгенерировать, например, рыжего ребенка в очках.

- при **большой размерности** латентное пространство может быть слишком **разреженным**.

При обучении модели количество точек в этом латентном пространстве будет настолько мало, что в основном пространство будет состоять из пустот. Тогда модель будет крайне некачественно генерировать объекты в точках латентного пространства, далеких от точек обучающей выборки.

Лучший способ **выбрать длину** вектора — это **найти публикацию с похожей задачей** и взять значение из нее. Если такой информации нет, то придется экспериментировать. Лучше **начинать** с **низкой размерности латентного пространства**, чтобы **наладить работу всей сети**, пусть и с низким разнообразием, а затем проводить эксперименты по **поиску оптимальной размерности**.

Можно использовать собственные знания в предметной области: спросите себя, **сколькими вещественными числами можно описать важную информацию об объекте**.

### Распределение входных латентных векторов

Как мы знаем из лекции про обучение сети, инициализация весов и нормализация входных данных вносят существенный вклад в работу модели. Поэтому принято использовать **многомерное нормальное распределение** для input latent space. Оно лучше взаимодействует с весами модели и улучшает сходимость.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L13/binomial_distribution.png" width="600"></center>

<center><em>Source: <a href="https://matplotlib.org/3.1.0/gallery/lines_bars_and_markers/scatter_hist.html">Двумерное нормальное распределение</a></em></center>

В этой лекции мы будем использовать `lightning` для обучения и `tbparse` для визуализации логов.

In [ ]:
from IPython.display import clear_output

!pip install lightning tbparse

clear_output()

Попробуем собрать **генератор точек неизвестной функции**. В качестве функции будем использовать **параболу**. Для начала напишем код, который будет создавать **истинные точки**:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def create_pair(num=100):
    x = np.random.uniform(low=-1, high=1, size=(num,))
    y = x * x
    return np.hstack(
        (x.reshape(-1, 1), y.reshape(-1, 1))
    )  # Create num of correct dots(x,y) on parabola


set_random_seed(42)
pairs = create_pair(100)
plt.scatter(pairs[:, 0], pairs[:, 1])
plt.title("Random dots on parabola,\nwhich will use like a dataset.")
plt.show()

Напишем датасет, который будет выдавать случайный шумовой вектор входного латентного пространства, размерностью  `ls = 1` (мы выбрали такую размерность т.к. знаем, что значения x и y связаны некоторой функцией) и истинные объекты.
Мы заранее создаем в функции  `__init__` точки, которые будем считать истинными.


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


# Define input parameters
n_batches = 30
batch_size = 128
ls = 1  # latent space

class ParabolaDS(Dataset):
    def __init__(self, n_batches, batch_size, ls):
        # create ground true item
        self.xy_pair = torch.tensor(
            create_pair(num=(n_batches * batch_size)), dtype=torch.float
        )
        self.ls = ls

    def __len__(self):
        return self.xy_pair.shape[0]

    def __getitem__(self, idx):
        # get ground true item
        item = self.xy_pair[idx]
        # generate random noise
        noise = torch.randn(self.ls, dtype=torch.float)
        return item, noise

trainset = ParabolaDS(n_batches, batch_size, ls)

train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

Определим простую модель, которая будет ожидать шум на вход и генерировать точки на выходе. Обратите внимание, что функция активации на последнем слое отсутствует, поскольку мы не ограничиваем наш генератор в каком-то диапазоне.

In [ ]:
import torch.nn as nn


class Generator(nn.Module):
    def __init__(self, latent_space, hidden_dim=50):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2),
        )  # x,y

    def forward(self, x):
        return self.model(x)

Напишем функцию для оценки loss.

Так как мы не знаем, в каком месте параболы генератор создаст новую точку, то непонятно, с каким элементом из датасета ее сравнивать.

**Вариант №1 (задача уже решена)**

Для сгенерированного $x$ аналитически вычислять $y_{target}=x*x$ и считать разницу между $y$, сгенерированным моделью, и $y_{target}$, вычисленным аналитически:

In [ ]:
def custom_loss(pair, label):
    # All inputs are batches
    x_fake = pair[:, 0]
    y_fake = pair[:, 1]
    return torch.abs(x_fake * x_fake - y_fake).mean()  # average by batch

Это будет работать.

Однако если мы **знаем способ точно предсказать выход по входу**, то **задача уже решена** и нейронная сеть не нужна.

**Вариант №2 (как не надо делать)**

Найти в датасете точку  $ \text{target} = (x_{\text{target}},y_{\text{target}})$, наиболее близкую к созданной генератором $ \text{generated} = (x,y)$, и использовать расстояние между этими точками в качестве loss.

$$\large \text{Loss} = \min(\text{dist}(\text{target}_{i},\text{generated}))$$


**В пространстве высокой размерности** такой поиск будет весьма **ресурсозатратным**, но в нашем учебном примере работать будет.


In [ ]:
class Loss(nn.Module):
    def __init__(self, targets):
        super().__init__()
        self.targets = targets  # Remember all real samples, impossible in real world

    def forward(self, input, dummy_target=None):
        dist = torch.cdist(input, self.targets)  # claculate pairwise distances (euc.)
        min_dist, index = torch.min(dist, dim=1)  # take the best
        return min_dist.mean()

Основной код обучения:

In [ ]:
import lightning as L

# DON'T USE IT! IT'S NAIVE DECISION!
class NaiveGAN(L.LightningModule):
    def __init__(self, model, criration):
        super().__init__()
        self.model = model
        self.criration = criration

    def configure_optimizers(self):
        self.criration.targets = self.criration.targets.to(self.device) # for fix device diff
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        real_items, noises = batch
        gen_items = self.model(noises)
        loss = self.criration(gen_items, real_items)
        self.log("loss/train", loss, on_epoch=True, on_step=False)
        return loss

Целевые точки из датасета запоминаются в loss, затем идет обычный цикл обучения:

In [ ]:
num_epochs = 10

criterion = Loss(trainset.xy_pair) # save target
model = Generator(latent_space=ls)

pl_model = NaiveGAN(model, criterion)
trainer = L.Trainer(
    max_epochs=num_epochs,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir="./log_naive/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader,
)

Посмотрим куда были сохранены логи:

In [ ]:
log_dir = pl_model.logger.log_dir
log_dir

Визуализируем кривую обучения, чтобы удостовериться, что модель учится:

In [ ]:
from tbparse import SummaryReader

def visualization(log_dir):
    # visualization without TensorBoard for TensorBoard logs
    clear_output()
    reader = SummaryReader(log_dir)
    df = reader.scalars.drop_duplicates()

    uniq = set(df.tag.unique())
    uniq.remove("epoch")
    uniq = list(uniq)
    uniq.sort()

    i = 0
    ax_dict = {}
    for item in uniq:
        metric = item.split("/")[0] # log shoud have tag
        if metric not in ax_dict:
            ax_dict[metric] = i
            i += 1

    fig, axs = plt.subplots(len(ax_dict), 1, figsize=(12, 3.5 * len(ax_dict)))
    for item in uniq:
        metric = item.split("/")[0]
        if len(ax_dict)>1:
            ax = axs[ax_dict[metric]]
        else:
            ax = axs
        sub_df = df[df["tag"] == item]
        ax.plot(sub_df.step, sub_df.value, label=item)
        ax.set_ylabel(metric)
        ax.legend()
        ax.set_xlabel("iter")

    plt.grid()
    plt.show()

visualization(log_dir)

Посмотрим результаты генерации на шуме:

In [ ]:
def test_image(pair_gen, pairs, figsize=None):
    if figsize:
        plt.figure(figsize=figsize)
    plt.scatter(pairs[:, 0], pairs[:, 1], label="real")
    plt.scatter(pair_gen[:, 0], pair_gen[:, 1], label="generated")
    plt.axis([-1, 1, 0, 1])
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid()
    plt.show()


model.eval().to("cpu")
noise = torch.tensor(np.random.normal(size=(1000, ls)), dtype=torch.float)
with torch.no_grad():
    pair_gen = model(noise).detach().numpy()

test_image(pair_gen, pairs)

Видно, что модель **генерирует точки**, лежащие на параболе, но при этом они лежат **в небольшом интервале около $0$**. Это можно объяснить, тем, что значения ошибок около нуля в среднем меньше, ведь $y$ около $0$ принимает небольшой диапазон значений.

В loss мы прописали, что **сгенерированная точка должна лежать на параболе**, и модель обучилась. Но информацию о том, в каких частях кривой должны оказаться точки, мы в loss никак не кодировали. Более того, модель может научиться хорошо генерировать одну единственную точку, и при этом loss может стать нулевым.

Кроме того, в loss сейчас **хранятся все известные точки**. Если бы мы генерировали не точки, а изображения или спектры - мы бы получили **переполнение оперативной памяти**.


Итак, надо решить **две проблемы**:

1.   Закодировать в loss условие о том, что **точки должны быть различными**.
2.   Придумать способ проверки, **не требующий хранения и перебора всего датасета**.



## Дискриминатор

Будем наказывать нейронную сеть не с помощью функционала ошибок , а второй
**сетью**, которая будет определять, лежит ли сгенерированная точка на параболе.

Создадим сеть-классификатор точек, которую назовём **дискриминатор**.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),  # real/fake
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

В общем случае задача дискриминатора — определять, **принадлежит ли объект к распределению обучающей выборки**. Если предсказания генератора выродятся в одно значение - дискриминатор легко запомнит, что эти точки - сгенерированные.


**Итого** мы имеем:
- **генератор**, выдающий точки, которые могут принадлежать параболе, а могут не принадлежать ей;
- **дискриминатор**, который будет их различать.

Мы будем подавать в **дискриминатор** **правильные точки**, помечая их реальными,  и **точки, которые выдаёт генератор**, считая их подделкой. **Генератор** будет учиться **подражать** реальным данным, а дискриминатор будет учиться **отличать** реальные точки, от подделок.

Мы пришли к идее **генеративно-состязательных** нейронных сетей (Generative Adversarial Network).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/generative_adversarial_network_scheme.png" width="700"></center>
<center><em>Схематичное представление архитектуры GAN </em></center>

##  Generative adversarial network (GAN)

[[arxiv] 🎓 Generative Adversarial Networks (Goodfellow et al., 2014)](https://arxiv.org/abs/1406.2661) (**Cited by 62647 (декабрь 2023)!!!**)

[[video] 📺 Разбор оригинальной статьи GAN](https://www.youtube.com/watch?v=eyxmSmjmNS0)

[[video] 📺 Лекция Иана Гудфеллоу](https://www.youtube.com/watch?v=HGYYEUSm-0Q)

**Генеративно-состязательную** сеть описал Иан Гудфеллоу из компании Google (на тот момент) в 2014 году. Принцип состязательности в сети GAN часто объясняют на примере из теории игр.

### GAN в теории игр

Математически можно описать идею GAN, как **[игру](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8%D0%B3%D1%80) двух игроков**:

 - Первый игрок - это **фальшивомонетчик** или генератор (generator).
 - Второй - это **банкир** с машинкой для проверки денег или дискриминатор (discriminator).


 Задача фальшивомонетчика - обмануть банкира. Задача банкира - отличить реальные деньги от фальшивых.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/gan_idea.png" width="600"></center>





Фальшивомонетчику необходим элемент случайности $z$. Можно привести аналогию: если все фальшивые купюры будут иметь одинаковый номер - банкиру будет очень просто определить подделку. Фальшивомонетчик берет на вход шум $z$ и генерирует фальшивую пачку денег $G(z)$.



Банкир $D$ получает на вход пачку купюр $x$, проверяет их подлинность и сообщает вектор $D(x)$, состоящий из чисел от нуля до единицы — свою уверенность (вероятность) по каждой купюре в том, что она настоящая.

**Цель банкира (дискриминатора):** отличить реальные деньги от фальшивых


$$D(x_i) = 1,  x_i  \text{ - real},$$

$$D(G(z_i)) = 0, G(z_i) \text{ - fake},$$


то есть максимизировать $log(D(x))+log(1-D(G(z)))$.

**Цель фальшивомонетчика (генератора):** убедить банкира в том, что подделки настоящие


$$D(G(z_i)) = 1, G(z_i) \text{ - fake}.$$


Математически это можно записать, как:

$$\large \min\limits_{\theta_g}  \max\limits_{\theta_d} [\mathbb{E}_{x _\sim p(x)} log(D_{\theta_d}(x)]+\mathbb{E}_{z _\sim p(z)}
[log(1-D_{\theta_d}(G_{\theta_g}(z))]$$

где $\theta_g$ - параметры генератора, в $\theta_d$ - параметры дискриминатора.


**Дискриминатор (банкир):**
- обучается при **фиксированном генераторе** ${G}_{\theta_{g}}$,
- **максимизирует** функцию выше относительно $\theta_d$ (**градиентный подъем**),
- решает задачу **бинарной классификации**: старается присвоить $1$ точкам данных из обучающего набора $E_{x∼p_{data}}$ и 0 сгенерированным выборкам $E_{z∼p(z)}$.

**Генератор (фальшивомонетчик):**
- обучается при **фиксированном дискриминаторе** $D_{θ_d}$,
- получает градиенты весов за счет backpropagation через дискриминатор,
- **минимизирует** функцию выше относительно $\theta_d$ (**градиентный спуск**).

В процессе совместного конкурентного обучения, если система достаточно сбалансирована, достигается **минимаксное состояние равновесия**, в котором обе сети эффективно учатся.


### GAN Практический пример

Определим наши **генератор** и **дискриминатор**:

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_space, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2),
        )  # x,y

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),  # real/fake
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

Определим **входные параметры**. Для обучения гана нужно много реальных объетов.

In [ ]:
latent_dim = 10  # latent space
num_epochs = 1
n_batches = 10000
batch_size = 32

Обратите внимание, что у нас так же, как и в первом примере, есть переменная **latent space**. Это тот шум, из которого мы будем генерировать наши точки.

**Дискриминатор** решает задачу **бинарной классификации**. Для этой задачи хорошо подходит [`BCELoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) (**Binary Cross Entropy**).

**Процесс обучения GAN**

Обучение дискриминатора:
1. Расчет градиентов дискриминатора (real точки):
    * обнулим градиенты **дискриминатора**
    * возьмем набор **real точек**, которые лежат на параболе,
    * применим к ним **дискриминатор**,
    * посчитаем значение функции потерь дискриминатора на **real точках** и **real метках** `loss_disc_real`,
    * посчитаем градиенты для **дискриминатора**.



<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/gan_training_algorithm_1.png" width="1000">

Продолжая аналогию с игрой: в этот момент банкир изучает реальные купюры, чтобы усовершенствовать машинку для проверки денег.

2. Расчет градиентов дискриминатора (fake точки):
    * возбмем случайный шум
$z$,
    * возьмем наш **генератор** (его пока не обучаем) и создадим с его помощью **fake точки** из $z$,
    * посчитаем значение функции потерь дискриминатора на **fake точках** и **fake метках** `loss_disc_fake`,
    * посчитаем градиенты для **дискриминатора** (они сложатся с уже посчитанными ранее).




<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/gan_training_algorithm_2.png" width="1000">


По аналогии с игрой: в этот момент банкир изучает фальшивые купюры, чтобы усовершенствовать машинку для проверки денег.

3. Обновление весов дискриминатора
   * сделаем шаг обучения **дискриминатора** (обновим его веса),
   * **генератор** не обучается.

По аналогии с игрой: в этот момент банкир модифицирует машинку.

Обучение генератора:

   * обнулим градиенты **генератора**,
   * сгенерируем случайный шум $z$,
   * создадим с помощью **генератора** набор **fake точек** из $z$,
   * посчитаем значение функции потерь дискриминатора на **fake точках** и **real метках** `loss_gen` (подмена меток),
   * посчитаем градиенты для **генератора**,
   * сделаем шаг обучения **генератора** (обновим его веса),
   * **дискриминатор** не обучается.

Обратите внимание, что вместо решения minmax задачи - мы подменяем метки. Это - [часто применяемый трюк](https://github.com/soumith/ganhacks/#2-a-modified-loss-function) при обучении GAN.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/gan_training_algorithm_3.png" width="1000">

По аналогии с игрой: фальшивомонетчик не просто генерирует подделки наудачу. Он добывает прибор для распознавания подделок, разбирает его, смотрит, как тот работает, и затем создает подделки, которые смогут обмануть этот прибор.

Посмотрим, как это реализуется в коде обучения:

In [ ]:
class GAN(L.LightningModule):
    def __init__(self, generator, discriminator, lr=3e-4, betas=(0.9, 0.999),
                 noise_in_place=False, latent_dim=10):
        super().__init__()
        self.automatic_optimization = False # for hand made settings

        self.generator = generator
        self.discriminator = discriminator
        self.criterion = nn.BCELoss()
        self.real_lable = 1.
        self.fake_lable = 0.
        self.lr = lr
        self.betas = betas
        self.noise_in_place = noise_in_place
        self.latent_dim = latent_dim

    def configure_optimizers(self):
        opt_gen = torch.optim.Adam(
            self.generator.parameters(),
            lr=self.lr,
            betas=self.betas,
        )
        opt_disc = torch.optim.Adam(
            self.discriminator.parameters(),
            lr=self.lr,
            betas=self.betas,
        )
        return [opt_gen, opt_disc], []

    def training_step(self, batch, batch_idx):
        if self.noise_in_place: # for standart dataset
            self.real_items, _ = batch
            noises = torch.randn(
                (self.real_items.shape[0], self.latent_dim),
                dtype=torch.float32,
            ).to(self.device)
        else:
            self.real_items, noises = batch # for heandmade dataset
        opt_gen, opt_disc = self.optimizers()

        # ---------------------
        # Train discriminator
        # ---------------------
        self.discriminator.zero_grad()
        # 1. discriminator on real items
        real_lable = torch.full(
            size=(self.real_items.shape[0], 1),
            fill_value=self.real_lable,
            dtype=torch.float,
        ).to(self.device)
        disc_lable = self.discriminator(self.real_items)
        loss_disc_real = self.criterion(disc_lable, real_lable)
        loss_disc_real.backward()

        # 2. discriminator on fake items
        fake_lable = torch.full(
            size=(self.real_items.shape[0], 1),
            fill_value=self.fake_lable,
            dtype=torch.float,
        ).to(self.device)
        self.fake_items = self.generator(noises)
        disc_lable = self.discriminator(self.fake_items)
        loss_disc_fake = self.criterion(disc_lable, fake_lable)
        loss_disc_fake.backward()

        # 3. discriminator optimizer step (on real and fake items)
        opt_disc.step()
        loss_disc = 0.5*loss_disc_real + 0.5*loss_disc_fake
        self.log("loss/disc", loss_disc, on_epoch=False, on_step=True)

        # ---------------------
        # Train generator
        # ---------------------
        self.generator.zero_grad()
        self.fake_items = self.generator(noises)
        disc_lable = self.discriminator(self.fake_items)
        loss_gen = self.criterion(disc_lable, real_lable)
        loss_gen.backward()

        opt_gen.step()
        self.log("loss/gen", loss_gen, on_epoch=False, on_step=True)

        if (batch_idx+1)%100==0:
            visualization(self.logger.log_dir)
            test_image(
                self.fake_items.detach().cpu().numpy(),
                self.real_items.detach().cpu().numpy(),
                figsize=(12, 3.5),
            )

    def on_train_epoch_end(self):
        visualization(self.logger.log_dir)
        test_image(
            self.fake_items.detach().cpu().numpy(),
            self.real_items.detach().cpu().numpy(),
            figsize=(12, 3.5),
        )

Создадим модели:

In [ ]:
set_random_seed(42)
trainset = ParabolaDS(n_batches, batch_size, latent_dim)

train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

generator = Generator(latent_space=latent_dim, hidden_dim=50)
discriminator = Discriminator(hidden_dim=50)

pl_model = GAN(generator, discriminator)

Обучим наш GAN:

In [ ]:
pl_model = GAN(generator, discriminator)
trainer = L.Trainer(
    max_epochs=num_epochs,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir="./log_gan/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader
)

У нас получилось сгенерировать точки равномерно распределенные по параболе.

## DCGAN — Генерация изображений

С помощью **GAN** можно, генерировать не только точки на параболе, но и, например, изображения. Но появляются закономерные вопросы.

### Как из шума на входе сети получить изображение?

Самым простым ответом будет: взять шум, пропустить его через **полносвязные слои** и сделать **reshape** до нужного разрешения. В целом, это будет работать.


Однако **DCGAN (Deep Convolutional GAN)** использует **сверточные** и **сверточно-транспонированные** (*convolutional* и *convolutional-transpose*) слои в дискриминаторе и генераторе соответственно. Впервые метод **DCGAN** был описан в статье [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks (Radford et al., 2015)](https://arxiv.org/abs/1511.06434).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/deep_convolutional_gan_scheme.png" width="700"></center>
<center><em>Схема работы DCGAN (Radford et al., 2015).</em></center>

Ниже видна разница в генерации при помощи исключительно **полносвязных слоёв** и при помощи **обратных свёрток**. Видно, что результат **DCGAN** лучше, чем **GAN**.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L13/gan_dcgan_mnist_examples.png" width="600"></center>
<center><em>Сравнение результатов на MNIST (Radford et al., 2015)</em></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1511.06434.pdf">	Unsupervised representation learning with deep convolutional generative adversarial networks</a></em></center>

### Архитектура DCGAN

На вход генератора подают шум для создания разнообразных объектов. Этот шум представляет собой вектор в многомерном пространстве. Один вектор — один сгенерированный объект. Задача генератора — преобразовать вектор в изображение.

Такое преобразование возможно при помощи транспонированных сверточных (convolution-transpose, иногда называют fractionally strided convolution) слоев или upsample слоев с последующей сверткой.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/dcgan_architecture.png" width="900"></center>
<center><em>Зеркальная архитектура DCGAN </em></center>

### Пример обученного DCGAN

Давайте посмотрим на пример обученного **DCGAN**

In [ ]:
use_gpu = True if torch.cuda.is_available() else False
model = torch.hub.load(
    "facebookresearch/pytorch_GAN_zoo:hub", "DCGAN", pretrained=True, useGPU=use_gpu
)
clear_output()

Распечатаем архитектуру генератора:

In [ ]:
print(model.getNetG())

Распечатаем архитектуру дискриминатора:

In [ ]:
print(model.getNetD())

In [ ]:
import torchvision


num_images = 16
noise, _ = model.buildNoiseData(num_images)
with torch.no_grad():
    generated_images = model.test(noise)
    generated_images = (
        generated_images.clamp(-1, 1) + 1
    ) / 2.0  # normalization to 0..1 range

fig, ax = plt.subplots(figsize=(16 * 3, 2 * 3))
ax.imshow(
    torchvision.utils.make_grid(generated_images).permute(1, 2, 0).cpu().numpy(),
    interpolation="nearest",
    aspect="equal",
)
ax.axis("off")
plt.show()

### Практический пример DCGAN

Теперь давайте попробуем сами написать свой **DCGAN** и обучить его на датасете **FashionMNIST**

In [ ]:
num_epochs = 3  # Num of epochs
batch_size = 64  # batch size
lr = 2e-4  # Learning rate
b1 = 0.5  # Adam: decay of first order momentum of gradient
b2 = 0.999  # Adam: decay of first order momentum of gradient
latent_dim = 100  # latent space
img_size = 32  # images size
channels = 1  # Num of channels

Обычно мы **инициализируем веса** случайным образом, но ничто не мешает нам инициализировать их так, как мы хотим. В [оригинальной статье](https://arxiv.org/pdf/1511.06434.pdf) про **DCGAN** предложено инициализировать веса нормальным распределением с центром в нуле и стандартным отклонением 0,02:

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)

Обратите внимание, как преобразуется **шум** в **генераторе**:
* Сначала с помощью **полносвязного слоя** он преобразуется в **первичные фичи**
* Потом с помощью функции `view` **ресэмплится** в картинку низкого разрешения
* Потом при прохождении через **conv_blocks** к нему применяются `ConvTranspose2d`

In [ ]:
class GeneratorBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.block = nn.Sequential(
            nn.ConvTranspose2d(
                in_channel, out_channel, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(out_channel),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class Generator(nn.Module):
    def __init__(self, latent_dim, img_size, channels):
        super(Generator, self).__init__()

        self.init_size = img_size // 2**2
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size**2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            GeneratorBlock(in_channel=128, out_channel=128),
            GeneratorBlock(in_channel=128, out_channel=64),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [ ]:
class DiscriminatorBlock(nn.Module):
    def __init__(self, in_channel, out_channel, bn=True):
        super().__init__()
        block = [
            nn.Conv2d(
                in_channel, out_channel, kernel_size=3, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
        ]
        if bn:
            block.append(nn.BatchNorm2d(out_channel, 0.8))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        return self.block(x)


class Discriminator(nn.Module):
    def __init__(self, channels):
        super().__init__()

        self.model = nn.Sequential(
            DiscriminatorBlock(channels, 16, bn=False),
            DiscriminatorBlock(16, 32),
            DiscriminatorBlock(32, 64),
            DiscriminatorBlock(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2**4
        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size**2, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [ ]:
set_random_seed(42)
L.seed_everything(42)

# Initialize Generator and Discriminator
generator = Generator(
    latent_dim=latent_dim, img_size=img_size, channels=channels
)
discriminator = Discriminator(channels=channels)

# Initialize weight
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

pl_model = GAN(generator, discriminator, noise_in_place=True, latent_dim=100,
               lr=lr, betas=(b1, b2))

Напишем функцию для отображения изображений

In [ ]:
from torchvision.utils import make_grid

def test_image(fake_items, real_items, figsize=(5, 5)):
    grid_fake = make_grid(
        torch.tensor(fake_items[:10]),
        nrow=10,
        normalize=True
    ).permute(1, 2, 0).numpy()

    grid_real = make_grid(
        torch.tensor(real_items[:10]),
        nrow=10,
        normalize=True
    ).permute(1, 2, 0).numpy()

    fig, ax = plt.subplots(2, 1, figsize=figsize)
    ax[0].imshow(grid_fake)
    ax[0].set_axis_off()
    ax[1].imshow(grid_real)
    ax[1].set_axis_off()
    plt.axis("off")
    plt.show()

Подгрузим данные и загрузим и их в Data Loader:

In [ ]:
import os
from torchvision import datasets, transforms

os.makedirs("../../data/mnist", exist_ok=True)
data_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [
                transforms.Resize(img_size),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),
            ]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)
clear_output()

In [ ]:
trainer = L.Trainer(
    max_epochs=num_epochs,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir="./log_cgan/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=data_loader
)

Мы проучили модель 3 эпохи. Видно, что Loss не только не сошлись, но начали расходиться. При этом на сгенерированных изображениях присутствует характерная рябь. Если увеличить количество эпох - качество генерируемых изображений улучшиться, эффект расхождения Loss только усилиться.


Такая ситуация происходит чаще, чем нам бы хотелось.  Ниже показаны кривые обучения простого GAN на датасете MNIST, взятые из [репозитория](https://github.com/znxlwm/pytorch-generative-model-collections/tree/master).

<center><img src ="https://raw.githubusercontent.com/znxlwm/pytorch-generative-model-collections/master/assets/mnist_results/GAN_loss.png" width="600"></center>
<center><em>Кривые обучения простого GAN на датасете MNIST</em></center>

<center><em>Source: <a href="https://github.com/znxlwm/pytorch-generative-model-collections"> Pytorch implementation of various GANs.</a></em></center>


## Тонкости обучения GANов

[Статья — детальный разбор тонкостей и советов](https://beckham.nz/2021/06/28/training-gans.html)

### Проблема больших градиентов

Одной из причин проблем со сходимостью GAN являются большие градиенты дискриминатора, которые приводят к сбою настроек генератора.

В статье [Wasserstein GAN](https://arxiv.org/pdf/1701.07875.pdf) показано, что обеспечение плавности ([k-Липшицевости](https://en.wikipedia.org/wiki/Lipschitz_continuity)) изменения параметров дискриминатора улучшает сходимость GAN. Есть несколько подходов к обеспечению такой плавности. Вот некоторые из них:
- Регуляризация:
    - **weight clipping** - клиппинг (ограничение максимального и минимального значения) весов. Может [давать чрезмерно сильную регуляризацию](https://beckham.nz/2021/06/28/training-gans.html);
    - **gradient penalty** - штраф за слишком большой или слишком маленький градиент (оптимальным выбран градиент с нормой 1). Cтатья [тут](https://arxiv.org/pdf/1704.00028.pdf). Код [тут](https://github.com/EmilienDupont/wgan-gp). Вычислительно сложная операция.
    - спектральная нормализация. [Вычислительно дешевый аналог gradient penalty](https://beckham.nz/2021/06/28/training-gans.html). Статья [тут](https://arxiv.org/pdf/1802.05957.pdf). Код - ниже.
- Изменение Loss:
    - функции c Cross-Entropy на **[расстояние Вассерштейна](https://en.wikipedia.org/wiki/Wasserstein_metric)** -  уход от неограниченно больших градиентов при уверенности дискриминатора, что сгенерированный объект - фейк. Применяется совместно с **weight clipping** или **gradient penalty**. Статья [тут](https://arxiv.org/pdf/1701.07875.pdf). Реализаця [тут](https://github.com/znxlwm/pytorch-generative-model-collections/blob/master/WGAN.py).
    - переход на сравнение статистик карт признаков.  Cтатья [тут](https://proceedings.neurips.cc/paper_files/paper/2016/file/8a3363abe792db2d8761d6403605aeb7-Paper.pdf). Реализаця [тут](https://github.com/openai/improved-gan).

- **Top-k Training:** отказ от обучения на плохих примерах. Градиент пропускается только через top-k наилучшим способом сгенеренных объектов. Объекты с большими градиентами отбрасываются. Статья [тут](https://arxiv.org/abs/2002.06224).


Мы попробуем применить к нашей модели спектральную нормализацию:

In [ ]:
from torch.nn.utils import spectral_norm

def add_spectral_norm(m):
    if isinstance(m, nn.Linear) or isinstance(m, torch.nn.Conv2d):
        m = spectral_norm(m)

In [ ]:
from torch.nn.utils import spectral_norm

set_random_seed(42)
L.seed_everything(42)

# Initialize Generator and Discriminator
generator = Generator(
    latent_dim=latent_dim, img_size=img_size, channels=channels
)

discriminator = Discriminator(channels=channels)

# Initialize weight
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

discriminator.apply(add_spectral_norm)

pl_model = GAN(generator, discriminator, noise_in_place=True, latent_dim=100,
               lr=lr, betas=(b1, b2))

In [ ]:
trainer = L.Trainer(
    max_epochs=num_epochs,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir="./log_cgan/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=data_loader
)

Loss меняется более устойчиво. Нет ряби на изображениях. При увеличении количества эпох - качество еще лучше.

### Другие особенности

[Репозиторий со списком советов по обучению GAN](https://github.com/soumith/ganhacks/)


**Простые ошибки**:
- Обратите внимание, что сгенерированные объекты должны быть в том же числовом диапазоне и иметь тот же размер, что и реальные данные.
- При использовании `BatchNorm` следите, чтобы  real и fake батчи [поступали на дискриминатор отдельно](https://github.com/soumith/ganhacks/?tab=readme-ov-file#4-batchnorm). При небольшом количестве объектов в батче или странном поведении модели в тестовом режиме - лучше использовать `InstanceNorm`.

**Нужно ли давать преимущество дискриминатору?**

Один из самых старых и распространенных советов при обучении GAN - давать преимущество дискриминатору. В большинстве случаев эту проблему может помочь решить ограниечение градиентов.



**Оптимизатор:**

В большинстве статьях **GAN** используется **ADAM**. В [статье](https://arxiv.org/pdf/1802.05957.pdf) проведенно исследование по поиску оптимальных параметров оптимизатора: для большинства архитектур $\beta_1 = 0.5, \beta_2 = 0.999, lr = 2\cdot 10^{-4}$ - хорошая отправная точка для обучения.

Параметр `epsilon` **ADAM** по умолчанию в PyTorch равен `1e-8`, что может вызвать проблемы после длительного периода обучения, например, значения функции потерь периодически взрываются или увеличиваются.  Для борьбы с этим, параметр `epsilon` можно увеличить, например до `1e-3`. Подробнее об этом на [StackOverflow](https://stackoverflow.com/questions/42327543/adam-optimizer-goes-haywire-after-200k-batches-training-loss-grows) и в комментарии на [Reddit](https://www.reddit.com/r/reinforcementlearning/comments/j9rflf/intuitive_explanation_for_adams_epsilon_parameter/).

## cGAN — GAN с условием

**cGAN** расшифровывается как **Conditional Generative Adversarial Net** — это **GAN** с условием. Условие может быть любым, например, генерация конкретной цифры. В этом случае нам нужен уже размеченный датасет для того, чтобы обучить Conditional GAN.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/conditional_gan_scheme.png" width="800"></center>
<center><em>Схема работы cGAN. Label Y добавляется к случайному шуму, тем самым мы говорим генератору генерировать случайное изображение нужного класса. Также он подаётся в дискриминатор в качестве входа, чтобы дискриминатор знал, какое изображение классифицировать как реальное, а какое — как вымышленное.</em></center>

Обучение в данном случае будет аналогичным обучению **GAN**: мы будем обучать сети, чередуя реальные данные и сгенерированные, добавив `label`.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L13/cGANS_results_20_and_50_epochs_mnist.png" width="600"></center>
<center><em>Сравнение результатов cGAN и cDCGAN.</em></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1511.06434.pdf">Unsupervised Representation Learning with Deep Convolutional Generative Adversial Networks</a></em></center>



### Как закодировать метки?

Поскольку подавать в сеть числа от 0 до 9 (в случае **MNSIT**) нет смысла, то нужно придумать, как подавать их в нейронную сеть. На помощь приходят **Embeddings**. Мы можем представить каждую метку в виде вектора с десятью элементами.

[Документация nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
samples, labels = next(iter(data_loader))

label_emb = nn.Embedding(10, 10)

e = label_emb(labels)

print(f"Label: {labels[0]}")
print(f"Embedding for this label: {e[0]}")

После этого **эмбеддинги** меток обычно склеиваются с входами сетей.

**Почему нельзя подать просто число?**

Если мы будем подавать просто число, например 0.1 для единицы, и 0.5 для пяти, то вход у нас будет непрерывным, что довольно нелогично: тогда при небольшом изменении входа мы будем генерировать другую цифру. А также сети будет сложнее выучить небольшие расхождения в этом небольшом интервале. В случае с векторным представлением мы избегаем этих проблем.

## Модификации GAN

Существует множество способов модификации GAN, например можно подмешивать метки классов только в дискриминатор, как сделано в [**Semi-Supervised GAN**](https://arxiv.org/pdf/1606.01583.pdf). В нем дискриминатор делает классификацию не на 2 класса **[real, fake]**, а на n+1 класс **[real 1, real 2, …, real n, fake]**, где n - количество классов в исходном датасете.

Другие модификации и пример кода с ними можно найти в [репозитории 1](https://github.com/znxlwm/pytorch-generative-model-collections) или [репозиотрии 2](https://github.com/eriklindernoren/PyTorch-GAN).

Также стоит отметить добавление модификаций исходного латентного пространства - **Style GAN**. Именно они позволили генерировать фотореалистичные изображения с  минимальным количеством артефактов. Примеры изображений можно посмотреть  [тут](https://nvlabs.github.io/stylegan3/).

[[arxiv] 🎓 A Style-Based Generator Architecture for Generative Adversarial Networks (StyleGAN) (Karras et al., 2018)](https://arxiv.org/abs/1812.04948)

[[arxiv] 🎓 Analyzing and Improving the Image Quality of StyleGAN (StyleGAN2) (Karras et al., 2019)](https://arxiv.org/abs/1912.04958)

[[arxiv] 🎓 Alias-Free Generative Adversarial Networks (Alias-Free GAN) (Karras et al., 2021)](https://arxiv.org/abs/2106.12423)


(TODO - РЕШИТЬ НУЖНО ЛИ ЭТО?)

**Недостатки GAN**
* **Нестабильное обучение**
* Очень **долгая сходимость**
* **Mode-collapsing** (модель выдает одно и то же изображение или один и тот же класс и т.д., независимо от того, какие входные данные ей подаются)
* **Исчезновение градиента**: дискриминатор настолько хорошо научился отличать сгенерированные образцы от реальных, что градиент весов генератора становится равным 0: в какую сторону бы генератор не изменил свои веса, дискриминатор все равно идеально распознает фальшивки
* Поиск оптимальных параметров — **pure luck**

# Диффузионные модели

Базовая публикация по теме: [Denoising Diffusion Probabilistic Models (Ho et al., 2020) 🎓[arxiv]](https://arxiv.org/abs/2006.11239).

## Прямой диффузный процесс

Пусть мы имеем некоторый объект $\mathbf{x}_0$ из заданного имеющимся датасетом распределения $\mathbf{x}_0 \sim q(\mathbf{x})$. Определим так называемый прямой диффузный процесс, в ходе которого мы будем последовательно добавлять небольшое количество Гауссового шума последовательно $T$ раз, создавая из нашего объекта последовательность $\mathbf{x}_1, \dots, \mathbf{x}_T$ постепенно всё более зашумленных версий нашего исходного объекта. Параметры добавляемого шума зависят от номера шага зашумления и их набор $\{\beta_t \in (0, 1)\}_{t=1}^T$ задан заранее.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/diffusion_process.png" width="700">

Исходное распределение данных из датасета будет последовательно преобразовано в Гауссов шум:

$$\large q(\mathbf{x}_t \vert \mathbf{x}_{t-1}) = \mathcal{N}(\mathbf{x}_t; \sqrt{1 - \beta_t} \mathbf{x}_{t-1}, \beta_t\mathbf{I}) \quad
q(\mathbf{x}_{1:T} \vert \mathbf{x}_0) = \prod^T_{t=1} q(\mathbf{x}_t \vert \mathbf{x}_{t-1})$$

Важным свойством описанного выше процесса является то, что он допускает аналитическое вычисление шума на любом из шагов процесса в явном виде. Это связано с тем, что сумма нескольких нормально распределенных случайных величин также является случайной величиной с нормальным распределением. Введём обозначения из оригинальной статьи $\bar{\alpha}_t = \prod_{i=1}^t \alpha_i$ и $\alpha_t = 1 - \beta_t$. Тогда:

$$ \large
\begin{aligned}
\mathbf{x}_t
&= \sqrt{\alpha_t}\mathbf{x}_{t-1} + \sqrt{1 - \alpha_t}\boldsymbol{\epsilon}_{t-1} \\
&= \sqrt{\alpha_t \alpha_{t-1}} \mathbf{x}_{t-2} + \sqrt{1 - \alpha_t \alpha_{t-1}} \bar{\boldsymbol{\epsilon}}_{t-2} \\
&= \dots \\
&= \sqrt{\bar{\alpha}_t}\mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_t}\boldsymbol{\epsilon} \\
\end{aligned} $$

$$ \large
\begin{aligned}
q(\mathbf{x}_t \vert \mathbf{x}_0) &= \mathcal{N}(\mathbf{x}_t; \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t)\mathbf{I})
\end{aligned}; $$

$\large \text{где } \boldsymbol{\epsilon}_{t-1}, \boldsymbol{\epsilon}_{t-2}, \dots \sim \mathcal{N}(\mathbf{0}, \mathbf{I}),$

$\large \qquad \bar{\boldsymbol{\epsilon}}_{t-2} \text{ – новая нормально распределенная величина (*)}.$

(*)  Напомним, что когда мы складываем две нормально распределенные величины с разной дисперсией и нулевым средним $\mathcal{N}(\mathbf{0}, \sigma_1^2\mathbf{I})$ и $\mathcal{N}(\mathbf{0}, \sigma_2^2\mathbf{I})$,то получаем новую нормально распределенную случайную величину $\mathcal{N}(\mathbf{0}, (\sigma_1^2 + \sigma_2^2)\mathbf{I})$, что в нашей параметризации означает $\sqrt{(1 - \alpha_t) + \alpha_t (1-\alpha_{t-1})} = \sqrt{1 - \alpha_t\alpha_{t-1}}$

## Обратный диффузный процесс

$$\large p_\theta(\mathbf{x}_{0:T}) = p(\mathbf{x}_T) \prod^T_{t=1} p_\theta(\mathbf{x}_{t-1} \vert \mathbf{x}_t) \quad
p_\theta(\mathbf{x}_{t-1} \vert \mathbf{x}_t) = \mathcal{N}(\mathbf{x}_{t-1}; \boldsymbol{\mu}_\theta(\mathbf{x}_t, t), \boldsymbol{\Sigma}_\theta(\mathbf{x}_t, t))$$

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L13/diffusion_example.png" width="700"></center>

<center><em>Source: <a href="https://lilianweng.github.io/posts/2021-07-11-diffusion-models">Lilian Weng: What are Diffusion Models?</a></em></center>

## Denoising U-Net

### Базовый блок

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/resnet_block.png" width="700">

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class ResNetBlock(nn.Module):
    def __init__(self, in_features, out_features, mid_features=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_features:
            mid_features = out_features
        self.conv_stack = nn.Sequential(
            nn.Conv2d(in_features, mid_features, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, mid_features),
            nn.GELU(),
            nn.Conv2d(mid_features, out_features, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, out_features),
        )

    def forward(self, x):
        if self.residual:
            return F.gelu(x + self.conv_stack(x))
        else:
            return self.conv_stack(x)

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/down_block.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/upblock.png" width="700">

In [ ]:
import torch


class ResizeBlock(nn.Module):
    def __init__(self, in_features, out_features, emb_dim):
        super().__init__()
        # defines non-linear map from time embedding features to conv features
        self.emb_projection = nn.Sequential(
            nn.SiLU(),
            nn.Linear(emb_dim, out_features),
        )

    def add_emb(self, x, t_vector):
        # [batch_size, time_embedding_dim] -> [batch_size, out_features]
        emb = self.emb_projection(t_vector)
        # [batch_size, out_features] - > [batch_size, out_features, H, W]
        emb = emb[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb

    def forward(self, x):
        pass


class Down(ResizeBlock):
    def __init__(self, in_features, out_features, emb_dim=256):
        super().__init__(in_features, out_features, emb_dim)
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            ResNetBlock(in_features, in_features, residual=True),
            ResNetBlock(in_features, out_features),
        )

    def forward(self, x, t):
        x = self.maxpool_conv(x)
        x = self.add_emb(x, t)
        return x


class Up(ResizeBlock):
    def __init__(self, in_features, out_features, emb_dim=256):
        super().__init__(in_features, out_features, emb_dim)

        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv = nn.Sequential(
            ResNetBlock(in_features, in_features, residual=True),
            ResNetBlock(in_features, out_features, in_features // 2),
        )

    def forward(self, x, skip_x, t):
        x = self.up(x)
        x = torch.cat([skip_x, x], dim=1)
        x = self.conv(x)
        x = self.add_emb(x, t)
        return x

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/out/se_block.png" width="700">

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super().__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, num_heads=4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.mlp = nn.Sequential(
            nn.LayerNorm([channels]),
            nn.Linear(channels, channels),
            nn.GELU(),
            nn.Linear(channels, channels),
        )

    def forward(self, x):
        # [batch_size, C, H, W] -> [batch_size, H*W, C]
        x = x.view(-1, self.channels, self.size * self.size).swapaxes(1, 2)
        x_ln = self.ln(x)
        # K, Q, V in Self attention are equal
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        # skip connection
        attention_value = attention_value + x
        # simple multilayer perceptron and second skip connection
        attention_value = self.mlp(attention_value) + attention_value
        # back to [batch_size, C, H, W]
        return attention_value.swapaxes(2, 1).view(
            -1, self.channels, self.size, self.size
        )

<img src ="https://dl.skoltech.ru/lectures/lecture-15/ddpm-architecture.png" width="700">

In [ ]:
class UNet(nn.Module):
    r"""
    Denoising U-Net model implementation based on arXiv:2006.11239 [cs.LG]
    """

    def __init__(self, num_channels=3, img_size=64, time_enbed_dim=256):
        super().__init__()
        self.num_channels = num_channels
        self.time_dim = time_enbed_dim
        self.img_size = img_size

        # Downsample and enlarge feature dim
        self.inc = ResNetBlock(num_channels, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128, img_size // 2)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256, img_size // 4)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256, img_size // 8)

        # Keep spatial dim constant
        self.conv_bottleneck = nn.Sequential(
            ResNetBlock(256, 512),
            SelfAttention(512, img_size // 8),
            ResNetBlock(512, 256),
        )

        # Upsample and reduce feature dim
        # 512=256+256 from conv_bottleneck and sa3
        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128, img_size // 4)
        # 256=128+128 from sa4 and sa2
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64, img_size // 2)
        # 128=64+64 from sa5 and sa1
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64, img_size)
        self.outc = nn.Conv2d(64, num_channels, kernel_size=1)

    def pos_encoding(self, t):
        r"""
        Returns embedding vector for given integer time index.

        We adopt 1d Positional Encoding form arXiv:1706.03762 [cs.CL]
        see 3.5 for more details.

        PE(x,2i) = sin(x/10000^(2i/D))
        PE(x,2i+1) = cos(x/10000^(2i/D))

        Where:
        x is a point in 1d space
        i is an integer in [0, D/2), where D is the size of the feature dimension

        Args:
            t: Tensor, shape ``[batch_size, 1]``
        Returns:
            pe: Tensor, shape ``[batch_size, time_embedding_dim]``
        """
        # placeholder for diffusion time encoding vector
        pe = torch.zeros(t.shape[0], self.time_dim).to(t.device)

        # factor 1/10000^(2i/D)
        div_factors = torch.exp(
            torch.arange(0, self.time_dim, 2)
            * (-torch.log(torch.as_tensor(10000.0)) / self.time_dim)
        ).to(t.device)

        pe[:, 0::2] = torch.sin(t * div_factors)
        pe[:, 1::2] = torch.cos(t * div_factors)

        return pe

    def forward(self, x, t):
        t = t.unsqueeze(-1).type(torch.float).to(x)
        t = self.pos_encoding(t)

        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        x4 = self.sa3(x4)

        x4 = self.conv_bottleneck(x4)

        x = self.up1(x4, x3, t)
        x = self.sa4(x)
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        output = self.outc(x)
        return output

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_noise_steps = 1000
time_enbed_dim = 256

model = UNet()

t = torch.arange(start=1, end=1001, step=1).unsqueeze(1)
positional_embeding = model.pos_encoding(t)
embeding_img = positional_embeding.numpy()

cax = plt.matshow(embeding_img[0:100])
plt.gcf().colorbar(cax)

## Реализация прямого и обратного диффузного процесса

In [ ]:
from IPython.display import clear_output
!pip install lightning
clear_output()

In [ ]:
import lightning as L
from torch.optim import AdamW
from torchvision.utils import make_grid
from PIL import Image
from tqdm.notebook import tqdm


L.seed_everything(42)

class DiffusionGenerativeModel(L.LightningModule):
    def __init__(
        self, model, path,
        num_noise_steps=1000,
        beta_start=1e-4,
        beta_end=0.02,
        img_size=64,
        lr=0.001,
        save_images=True
    ):
        super().__init__()
        self.model = model
        self.path = path
        self.num_noise_steps = num_noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.lr = lr
        self.save_images = save_images

        # diffusion process linear noise schedule
        self.beta = self._get_noise_schedule()
        self.alpha = 1.0 - self.beta
        # \hat{\alpha}_{i-1} = \prod_{j=0}^{i-1} \alpha_j
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

        # init optimizer and loss for training
        self.criterion = nn.MSELoss()

    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.lr)
        return optimizer

    def _get_noise_schedule(self):
        return torch.linspace(
            self.beta_start, self.beta_end, self.num_noise_steps
        )

    def _noise_images_batch(self, x, t):
        # \mu_i = \sqrt{\hat{\alpha}_i}
        mu = torch.sqrt(self.alpha_hat[t])[:, None, None, None].to(self.device)
        # \sigma_i = \sqrt{1 - \hat{\alpha}_i}
        sigma = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None].to(self.device)

        standard_normal_noise = torch.randn_like(x).to(self.device)
        noised_image_batch = mu * x + sigma * standard_normal_noise
        return noised_image_batch, standard_normal_noise

    def _get_timestams_batch(self):
        return torch.randint(
            low=1, high=self.num_noise_steps, size=(self.batch_size,)
        )

    def generate_images_batch(self):
        # start from pure noise batch
        x = torch.randn((self.batch_size, 3, self.img_size, self.img_size)).to(self.device)
        # and apply self.num_noise_steps denoising steps with model
        for t_i in tqdm(reversed(range(1, self.num_noise_steps)), position=0):
            # Build tensor with timestamp index. Same for each element in batch
            t = torch.full((self.batch_size,), t_i).long().to(self.device)

            # predict noise on current timestamp
            with torch.inference_mode():
                pred_noise = self.model(x, t).detach()

            # restore noise parametrs on current timestamp
            alpha = self.alpha[t.to('cpu')][:, None, None, None].to(self.device)
            alpha_hat = self.alpha_hat[t.to('cpu')][:, None, None, None].to(self.device)
            beta = self.beta[t.to('cpu')][:, None, None, None].to(self.device)

            # partialy denoise batch of images
            x = x - (1.0 - alpha) / (1 - alpha_hat).sqrt() * pred_noise
            x = (1 / alpha.sqrt()) * x

            # add appropriate amount of noise for next step if any
            if t_i > 0:
                z = torch.randn_like(x).to(self.device)
                x = x + beta.sqrt() * z
        # clip x to valid 0..255 image range
        x = x.clamp(0, 1)
        x = (x * 255).type(torch.uint8)
        return x

    def training_step(self, batch, batch_idx):
        # unpack data
        images, labels = batch
        self.batch_size = images.shape[0]
        t = self._get_timestams_batch()

        # prep batch of noised images
        noised_images, target_noise = self._noise_images_batch(images, t)

        # estimate noise with U-Net
        predicted_noise = self.model(noised_images, t)

        # optimize model to fit target noise
        loss = self.criterion(predicted_noise, target_noise)
        self.log("loss", loss, prog_bar=True, on_epoch=True, on_step=False)

        return loss

    def on_train_epoch_end(self):
        if self.save_images:
            generated_images = self.generate_images_batch()
            image_grid = make_grid(generated_images)
            # convert to numpy
            ndarr = image_grid.permute(1, 2, 0).to("cpu").numpy()
            # and save
            im = Image.fromarray(ndarr)
            im.save(f"{self.path}model_{self.current_epoch}.jpg")

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        images, labels = batch
        self.batch_size = images.shape[0]
        return self.generate_images_batch()

## Обучение диффузных моделей

In [ ]:
!pip install -q astronn

Рассмотрим пример использования опеределенной выше диффузной модели генерации на примере датасета Galaxy10

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L13/galaxy10sdss_example.png" width="800"></center>

<center><em>Примеры каждого класса изображений из датасета Galaxy10</a></em></center>

<center><em>Source: <a href="https://github.com/henrysky/Galaxy10">Galaxy10 DECals Dataset</a></em></center>


In [ ]:
from astroNN.datasets import load_galaxy10

images, labels = load_galaxy10()

In [ ]:
from torch.utils.data.dataset import Dataset


class GalaxyDataset(Dataset):
    def __init__(self, images, labels, transform):
        super().__init__()
        self.images = images
        self.labels = labels
        self.transform = transform

    def __getitem__(self, indx):
        image = self.images[indx]
        label = self.labels[indx]

        if self.transform:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [ ]:
from torchvision import transforms


transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(64, antialias=True),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomVerticalFlip(0.5),
    ]
)

dataset = GalaxyDataset(images, labels, transform)

Обучение сети осуществляется следующим образом. Пропустим этот этап, так как процесс занимает продолжительное время.

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint

path = "/diffusion/"

train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=8, shuffle=True, drop_last=True, num_workers=2
)

checkpoint_callback = ModelCheckpoint(
    dirpath=path,
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    monitor="loss",
    filename="best",
    mode="min",
)

model = UNet()

ddpm = DiffusionGenerativeModel(model, path)

trainer = L.Trainer(
    max_epochs=200,
    callbacks=[checkpoint_callback],
    log_every_n_steps=100,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir=path),
)

#trainer.fit(model=ddpm, train_dataloaders=train_loader)

Посмотрим на изображения, сгенерированные необученной моделью:

In [ ]:
dataset = GalaxyDataset(images[:16], labels[:16], transform)

gen_loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=False, num_workers=2
) # just for image num

generated_images = trainer.predict(ddpm, gen_loader)[0]

image_grid = make_grid(generated_images)
# convert to numpy
ndarr = image_grid.permute(1, 2, 0).to("cpu").numpy()
# and show
Image.fromarray(ndarr)

Видим, что сгенерированные изображения представляют собой шум.

Загрузим веса предобученной модели:


In [ ]:
! wget -q https://edunet.kea.su/repo/EduNet-content/dev-2.0/L13/weights/unconditioned_baseline.pt

In [ ]:
model = UNet()
model.load_state_dict(
    torch.load("/content/unconditioned_baseline.pt", map_location="cpu")
)
ddpm = DiffusionGenerativeModel(model, path)

In [ ]:
generated_images = trainer.predict(ddpm, gen_loader)[0]
image_grid = make_grid(generated_images)
# convert to numpy
ndarr = image_grid.permute(1, 2, 0).to("cpu").numpy()
# and show
Image.fromarray(ndarr)

<font size="6">Использованная литература</font>

<font size="5">GAN</font>

[[book] 📚 «Генеративное глубокое обучение. Творческий потенциал нейронных сетей»](https://habr.com/ru/company/piter/blog/504956/)

[[arxiv] 🎓 Generative Adversarial Networks (Goodfellow et al., 2014)](https://arxiv.org/abs/1406.2661)

[[video] 📺 Разбор оригинальной статьи GAN](https://www.youtube.com/watch?v=eyxmSmjmNS0)

[[video] 📺 Лекция Иана Гудфеллоу](https://www.youtube.com/watch?v=HGYYEUSm-0Q)

[Generative adversarial networks](https://deepgenerativemodels.github.io/notes/gan/)

[Самые современные генеративные модели](https://paperswithcode.com/methods/category/generative-models)

[exactly how the NVIDIA GauGAN neural network works](https://sudonull.com/post/29972-Pictures-from-rough-sketches-how-exactly-the-NVIDIA-GauGAN-neural-network-works-ITSumma-Blog)

<font size="5">DCGAN</font>

[[arxiv] 🎓 Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks (Radford et al., 2015)](https://arxiv.org/abs/1511.06434).

[DCGAN TUTORIAL](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

<font size="5">Wasserstein GAN</font>

[[arxiv] 🎓 Wasserstein GAN (Arjovsky et. al., 2017)](https://arxiv.org/abs/1701.07875)

[Блог пост о Wasserstein GAN](https://vincentherrmann.github.io/blog/wasserstein/)

[[arxiv] 🎓 Improved Training of Wasserstein GANs (Gulrajani et al., 2017)](https://arxiv.org/abs/1704.00028)

[[arxiv] 🎓 Spectral Normalization for Generative Adversarial Networks (Miyato et al., 2018)](https://arxiv.org/abs/1802.05957).

<font size="5">ProGAN -> StyleGAN -> StyleGAN2 -> Alias-Free GAN</font>

[[arxiv] 🎓 Progressive Growing of GANs for Improved Quality, Stability, and Variation (ProGAN) (Karras et al., 2017)](https://arxiv.org/abs/1710.10196)

[[arxiv] 🎓 A Style-Based Generator Architecture for Generative Adversarial Networks (StyleGAN) (Karras et al., 2018)](https://arxiv.org/abs/1812.04948)

[[arxiv] 🎓 Analyzing and Improving the Image Quality of StyleGAN (StyleGAN2) (Karras et al., 2019)](https://arxiv.org/abs/1912.04958)

[[arxiv] 🎓 Alias-Free Generative Adversarial Networks (Alias-Free GAN) (Karras et al., 2021)](https://arxiv.org/abs/2106.12423)

<font size="5">Тонкости обучения GAN</font>

[[blog] ✏️ Детальный разбор тонкостей и советов](https://beckham.nz/2021/06/28/training-gans.html)

[[arxiv] 🎓 Top-k Training of GANs: Improving GAN Performance by Throwing Away Bad Samples (Sinha et al., 2020)](https://arxiv.org/abs/2002.06224)

<font size="5">GAN Zoo:</font>

<font size="5">BigGAN</font>

[[arxiv] 🎓 Large Scale GAN Training for High Fidelity Natural Image Synthesis (Brock et al., 2018)](https://arxiv.org/abs/1809.11096)

<font size="5">StackGAN</font>

[StackGAN: Text to Photo-realistic Image Synthesis with Stacked Generative Adversarial Networks (Zhang et al., 2016)](https://arxiv.org/abs/1612.03242)

Дополнительно: [репозиторий к статье с воспроизведением результатов](https://github.com/hanzhanggit/StackGAN).

[[arxiv] 🎓 StackGAN++: Realistic Image Synthesis with Stacked Generative Adversarial Networks (Zhang et al., 2017)](https://arxiv.org/abs/1710.10916)

[[arxiv] 🎓 Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network (Ledig et al., 2016)](https://arxiv.org/abs/1609.04802)

[Deep Learning Generative Models for Image Synthesis and Image Translation](https://www.rulit.me/data/programs/resources/pdf/Generative-Adversarial-Networks-with-Python_RuLit_Me_610886.pdf)

[[video] 📺 StackGAN++ Realistic Image Synthesis with Stacked Generative Adversarial Networks | AISC](https://www.youtube.com/watch?v=PXWIaLE7_NU)

[[video] 📺 Text to Photo-realistic Image Synthesis with Stacked Generative Adversarial Networks](https://www.youtube.com/watch?v=crI5K4RCZws)

<font size="5">ControlGAN</font>

[[arxiv] 🎓 Controllable Generative Adversarial Network](https://arxiv.org/pdf/1708.00598.pdf)

[[arxiv] 🎓 Controllable Text-to-Image Generation](https://arxiv.org/pdf/1909.07083.pdf)

[[arxiv] 🎓 Image Generation and Recognition (Emotions)](https://arxiv.org/pdf/1910.05774.pdf)

[Natural Language & Text-to-Image 2019](https://meta-guide.com/data/data-processing/text-to-image-systems/natural-language-text-to-image-2019)

<font size="5">AC-GAN</font>

[How to Develop an Auxiliary Classifier GAN (AC-GAN) From Scratch with Keras](https://machinelearningmastery.com/how-to-develop-an-auxiliary-classifier-gan-ac-gan-from-scratch-with-keras/)

Conditional Image Synthesis with Auxiliary Classifier GANs: [статья](https://arxiv.org/pdf/1610.09585.pdf), [репозиторий](https://github.com/clvrai/ACGAN-PyTorch)

[[arxiv] 🎓 An Auxiliary Classifier Generative Adversarial Framework for Relation Extraction (Zhao, 2019)](https://arxiv.org/pdf/1909.05370.pdf)

[A Multi-Class Hinge Loss for Conditional GANs](https://openaccess.thecvf.com/content/WACV2021/papers/Kavalerov_A_Multi-Class_Hinge_Loss_for_Conditional_GANs_WACV_2021_paper.pdf)

<font size="5">Domain Transfer Network</font>

[[arxiv] 🎓 Unsupervised Cross-Domain Image Generation (Taigma et al., 2016)](https://arxiv.org/abs/1611.02200)

<font size="5">Pix2Pix</font>

[[arxiv] 🎓 Image-to-Image Translation with Conditional Adversarial Networks (Isola et al., 2016)](https://arxiv.org/abs/1611.07004)

<font size="5">Семантическая генерация</font>

[[arxiv] 🎓 Learning to Generate Chairs, Tables and Cars
with Convolutional Networks (Dosovitskiy et al., 2017)](https://arxiv.org/abs/1411.5928)

<font size="5">Text to image</font>

[[arxiv] 🎓 Text-to-Image Generation with Attention Based Recurrent Neural Networks (Zia et al., 2020)](https://arxiv.org/abs/2001.06658)

<font size="5">Image-to-Image</font>

[[arxiv] 🎓 GANs N' Roses: Stable, Controllable, Diverse Image to Image Translation (Chong et al., 2021)](https://arxiv.org/abs/2106.06561)

[[arxiv] 🎓 Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks (Zhu et al., 2017)](https://arxiv.org/abs/1703.10593)

<font size="5">Ссылки</font>

[[git] 🐾 MNIST CelebA cGAN cDCGAN](https://github.com/znxlwm/pytorch-MNIST-CelebA-cGAN-cDCGAN)

[[git] 🐾 Text-to-Photo realistic Image Synthesis with Stacked Generative Adversarial Networks](https://github.com/zeusm9/Text-to-Photo-realistic-Image-Synthesis-with-Stacked-Generative-Adversarial-Networks)

[[git] 🐾 ControlGAN](https://github.com/mrlibw/ControlGAN)

[[git] 🐾 ControlGAN-Tensorflow](https://github.com/taki0112/ControlGAN-Tensorflow)

[[git] 🐾 Keras-ACGan](https://github.com/lukedeo/keras-acgan)

[Множество примеров различных генераторов](https://thisxdoesnotexist.com)
